# Зависимость между двумя категорийными переменными
Для изучения процесса выявления зависимости между двумя категорийными переменными создан отдельный набор данных **npi_sample.csv**

Данный набор даных представляет собой опрос с 5-ю вопросами

Колонки набора данных (тип данных всех значений набора данных являются логическими, слева ответ положительный справа отрицательный):
- influence - У меня есть талант влиять на людей / Я не очень хорошь во влиянии на людей
- blend_in - Я предпочитаю сливаться с толпой / Я люблю быть в центре внимания
- special - Я считаю себя особенным / Я не лучше и не хуже большинства людей
- leader - Я считаю себя хорошим лидером / Я не уверен смог бы я стать хорошим лидером
- authority - Мне нравится иметь авторитет над людьми / Я не против выполнять приказы

Как можно догадаться ответы на некоторые вопросы могут быть связаны между собой. Например если кто-то считается себя хорошим лидером, мы могли бы заметить что также этот человек отмечал что он любить быть авторитетом 

Подробнее рассмотрим расчет показателей зависимостей между двумя категорийными переменными

In [1]:
# Импортируем библиотеки
import pandas as pd

# Загрузим данные из файла
npi = pd.read_csv('npi_sample.csv')
# отобразим несколько первых строк
print(npi.head())

  influence blend_in special leader authority
0        no      yes     yes    yes       yes
1        no      yes      no     no        no
2       yes       no     yes    yes       yes
3       yes       no      no    yes       yes
4       yes      yes      no    yes        no


In [7]:
# Для расчета показателя формируется Таблица непредвиденных обстоятельств: частота (Contingency Tables: Frequencies)
# Такую таблицу также называют кросс таблицей (cross-tabulations)
# Такие таблицы очень полезы для получения сводной информации двух переменных одновременно
# Чтобы создать такую таблицу используется метод библиотеки pandas crosstab
# Данный метод принмиает два массивы данных
print(pd.crosstab(npi.special, npi.authority))

authority    no   yes
special              
no         4069  1905
yes        2229  2894


## Анализ таблицы
Таблица показывает нам какое кол-во опрошенных людей:
- ответили Да на оба вопроса
- ответили Нет на оба вопроса
- ответили Да на один из них

Судя по значениям можно заметить что:
- наибольшее кол-во людей ответили Нет на оба вопроса
    - Люди не любят иметь авторитет
    - И не считают себя особенными
- И во второй колонке люди которые любят иметь авторитет считают себя особенными


In [8]:
# Но на основе цифр не всегда можно увидеть явную картину
# В таком случае используют пропорции
# Для того чтобы получить таблицу пропорций следует просто поделить каждое значение полученной таблицы поделить на общее кол-во записей в таблице
print(pd.crosstab(npi.special, npi.authority) / len(npi))

authority        no       yes
special                      
no         0.366676  0.171668
yes        0.200865  0.260791


## Маржинальная пропорция
По мимо вычисления обычной пропорции также для выявления зависимости расчитывают и маржинальную пропорцию

### Что такое маржинальная пропорция?
Маржинальной пропорцией называют сумму значений всех строк и столбцов по каждой категории

Сделать это можно используя метод суммы sum() с параметром axis = 1\0
- где 0 это столбцы
- а 1 то строки

In [ ]:
special_authority_prop = pd.crosstab(npi.special, npi.authority) / len(npi)
authority_marginal = special_authority_prop.sum(axis=0)
special_marginal = special_authority_prop.sum(axis=1)
print(authority_marginal, special_marginal, sep='\n')

# Маржинальные пропорции каждой категории с ответами приблизительно равны
# Что может сказать о наличии зависимости между двумя данными параметрами

authority
no     0.567541
yes    0.432459
dtype: float64
special
no     0.538344
yes    0.461656
dtype: float64


## Ожидаемая таблица вероятностей
Ранее созданная таблица маржинальных пропорций в последующем будет использована для создания ожидаемой таблицы вероятностей

Данная таблица позволит нам понят а существует ли вообще взаимосвязь между двумя категорийными переменными

Для формирования данной таблицы используется отдельный метод библиотеки **scipy.stats** **chi2_contingency**

Который принимает на вход таблицу возможных значений (crosstable)

Данный метод возвращает кортеж из трех значений:
- chi2 - параметр который информирует насколько сильно отличаются значения анализируемой таблицы и ожидаемой
- pval
- dof
- expected - таблица ожидаемых вероятностных значений

In [19]:
# загружаем метод из библиотеки
from scipy.stats import chi2_contingency
import numpy as np

special_authority_freq = pd.crosstab(npi.special, npi.authority)

chi2, pval, dof, expected = chi2_contingency(special_authority_freq)
print(special_authority_freq)
print(np.round(expected))

# Чем более различные ожидаемые и анализируемые значения в таблицах
# Тем БОЛЕЕ связанными являются переменные

authority    no   yes
special              
no         4069  1905
yes        2229  2894
[[3390. 2584.]
 [2908. 2215.]]


In [20]:
# В зависимости от размера иследуемой таблицы (в нашем случае 2 х 2 = 4)
print(chi2)
# Чем больше значение chi2 чем площадь исследуемой таблицы тем сильнее зависимость между иследумыми параметрами

679.1219526170606
